In [ ]:
import pandas as pd
import requests
import io


def urlopen(url):
    try:
        response = requests.get(url, allow_redirects=True)
    except requests.exceptions.ConnectionError:
        return None
    try:
        response.raise_for_status()
    except requests.exceptions.HTTPError:
        return None
    data = io.BytesIO(response.content)
    data.seek(0)
    return data


def parse_results(url, key, protocol="tabledap"):
    data = urlopen(url)
    if data is None:
        return None
    df = pd.read_csv(data)
    try:
        df.dropna(subset=[protocol], inplace=True)
    except KeyError:
        return None
    return {key: df[["Title", "Institution", "Dataset ID"]]}

In [ ]:
from erddapy import servers

query = "glider"

urls = {
    key: f'{server.url}search/index.csv?page=1&itemsPerPage=100000&searchFor="{query}"'
    for key, server in servers.items()
}


In [ ]:
from joblib import Parallel, delayed
import multiprocessing


num_cores = multiprocessing.cpu_count()

In [ ]:
returns = Parallel(n_jobs=num_cores)(
    delayed(parse_results)(url, key, protocol="tabledap") for key, url in urls.items()
)

In [ ]:
dfs = [x for x in returns if x is not None]

In [ ]:
[list(df.keys())[0] for df in dfs]

In [ ]:
pd.concat([list(dfs[0].values())[0] for df in dfs])